                Data Engineering for "percentage by state -> deaths and incidents count"

In [2]:
# Dependancies
import pandas as pd
import os
import csv

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base

# Define Declarative Base 
Base = declarative_base()

In [3]:
# Read in CSV File
df=pd.read_csv('db/percentage_by_state.csv', dtype={'Zip': 'str'})
df.head(20)

,state,abr,lat,lng,CancerType,incidence,population,percentage_incident,prevalence,alive,year,death_count,prevalence_1_year,percentage_deaths,diff
0,Alabama,AL,32.601011,-86.680736,Female Breast,18227,12434320,0.146586,"0,15",1865148,2011-2015,3394,"0,150272954",0.027295,0
1,Alaska,AK,61.302501,-158.775020,Female Breast,2100,1751388,0.119905,"0,12","210166,56",2011-2015,310,"0,120177254",0.017700,0
2,Arizona,AZ,34.168219,-111.930907,Female Breast,22214,16686987,0.133122,"0,13","2169308,31",2011-2015,3957,"0,130237112",0.023713,0
3,Arkansas,AR,34.751928,-92.131378,Female Breast,10399,7527040,0.138155,"0,14","1053785,6",2011-2015,2050,"0,140272405",0.027235,0
4,California,CA,37.271875,-119.270415,Female Breast,128452,96473279,0.133148,"0,13","12541526,27",2011-2015,22033,"0,130228382",0.022838,0
5,Colorado,CO,38.997934,-105.550567,Female Breast,17933,13143731,0.136438,"0,14","1840122,34",2011-2015,2796,"0,140212699",0.021272,0
6,Connecticut,CT,41.518784,-72.757507,Female Breast,16000,9201326,0.173888,"0,17","1564225,42",2011-2015,2288,"0,170248614",0.024866,0
7,Delaware,DE,39.145251,-75.418921,Female Breast,3994,2387966,0.167255,"0,17","405954,22",2011-2015,672,"0,170281319",0.028141,0
8,District of C,DC,38.899349,-77.014567,Female Breast,2350,1700582,0.138188,"0,14","238081,48",2011-2015,509,"0,140299027",0.029931,0
9,Florida,FL,27.975728,-83.833017,Female Breast,77149,50178974,0.153748,"0,15","7526846,1",2011-2015,14200,"0,150282985",0.028299,0


In [4]:
# Inspecting the Data
df.shape

(50, 15)

In [5]:
# Check columns for missing data
df.isnull().sum()

state                  0
abr                    0
lat                    0
lng                    0
CancerType             0
incidence              0
population             0
percentage_incident    0
prevalence             0
alive                  0
year                   0
death_count            0
prevalence_1_year      0
percentage_deaths      0
diff                   0
dtype: int64

In [6]:
# Get min and max (for leaflet part)
print(df['percentage_deaths'].min())
print(df['percentage_deaths'].max())

0.017700247
0.031030538


In [7]:
# Drop unnessesary columns. NOTE: data has no Missing/NaN values
dropped_df = df.drop(['CancerType', 'prevalence', 'alive', 'year', 'prevalence_1_year', 'diff'], axis=1)
dropped_df.head(2)

,state,abr,lat,lng,incidence,population,percentage_incident,death_count,percentage_deaths
0,Alabama,AL,32.601011,-86.680736,18227,12434320,0.146586,3394,0.027295
1,Alaska,AK,61.302501,-158.775020,2100,1751388,0.119905,310,0.017700


In [8]:
# Creating an engine, sqlite database and connection to the engine
engine = create_engine("sqlite:///breast_cancer.sqlite", echo=False)
conn = engine.connect()

In [9]:
# Creating a Class & table 
class Leads(Base):
    __tablename__ = 'states_percentage_table'
    id = Column(Integer, primary_key=True)
    state = Column(String(255))
    abr = Column(String(255))
    lat = Column(Float)
    lng = Column(Float)
    incidence = Column(Float)
    population = Column(Float)
    percentage_incident = Column(Float)
    death_count = Column(Float)
    percentage_deaths = Column(Float)   

In [10]:
Base.metadata.create_all(conn)

In [11]:
# Transferring the data from dataframe to the sqlite db
dropped_df.to_sql(name='states_percentage_table', con=conn, if_exists='append', index=False)

In [12]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['states_percentage_table']

In [13]:
# Checking the data 
engine.execute('SELECT * FROM states_percentage_table LIMIT 20').fetchall()

[(1, 'Alabama', 'AL', 32.601011199999995, -86.6807365, 18227.0, 12434320.0, 0.14658622300000002, 3394.0, 0.027295421),
 (2, 'Alaska', 'AK', 61.3025006, -158.7750198, 2100.0, 1751388.0, 0.11990489800000001, 310.0, 0.017700247),
 (3, 'Arizona', 'AZ', 34.1682185, -111.930907, 22214.0, 16686987.0, 0.133121695, 3957.0, 0.023713089),
 (4, 'Arkansas', 'AR', 34.7519275, -92.13137840000002, 10399.0, 7527040.0, 0.138155238, 2050.0, 0.027235142),
 (5, 'California', 'CA', 37.2718745, -119.2704153, 128452.0, 96473279.0, 0.13314775, 22033.0, 0.022838447999999997),
 (6, 'Colorado', 'CO', 38.9979339, -105.55056699999999, 17933.0, 13143731.0, 0.136437668, 2796.0, 0.021272499),
 (7, 'Connecticut', 'CT', 41.518783500000005, -72.75750699999999, 16000.0, 9201326.0, 0.17388798100000002, 2288.0, 0.024865981000000002),
 (8, 'Delaware', 'DE', 39.145251, -75.41892059999999, 3994.0, 2387966.0, 0.16725531300000002, 672.0, 0.028141104),
 (9, 'District of C', 'DC', 38.899348700000004, -77.0145666, 2350.0, 1700582.0